<a href="https://colab.research.google.com/github/gorkham8/Name-Generator/blob/main/NameGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np

def load_doc(filename):
  file = open(filename,'r')
  text = file.read()
  file.close()
  return text


def remove_after_first_space(input_string):
    return input_string.split(' ', 1)[0]

raw_text = load_doc("drive/MyDrive/Colab/nameData.txt")
tokens = raw_text.split()
raw_text = ' '.join(tokens)

length = 3

def pad_with_spaces(input_string):
    return input_string.ljust(length+1)

sequences = list()
for i in range(length, len(raw_text)):
  seq = raw_text[i-length:i+1]
  sequences.append(seq)
print('Total Sequences: %d'% len(sequences))

for i in range(len(sequences)):
  strn = remove_after_first_space(sequences[i])
  sequences[i] = pad_with_spaces(strn)

sequences2 = [seq for seq in sequences if not seq.startswith(' ')]
sequences = sequences2

def save_doc(lines,filename):
  data = '\n'.join(lines)
  file = open(filename,'w')
  file.write(data)
  file.close

out_filename = 'drive/MyDrive/Colab/nameData_sequences.txt'
save_doc(sequences,out_filename)

in_filename='drive/MyDrive/Colab/nameData_sequences.txt'
raw_text = load_doc(in_filename)
lines = raw_text.split('\n')
print(lines)
#encoding sequence
chars = sorted(list(set(raw_text)))
mapping = dict((c,i)for(i,c)in enumerate(chars))
print(mapping)

sequences = list()
for line in lines:
 # integer encode line
 encoded_seq = [mapping[char] for char in line]
 # store
 sequences.append(encoded_seq)
print(sequences)
vocab_size = len(mapping)

sequences = np.asarray(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
print(X.shape)
print(y.shape)
print(X)
print(y)
from keras.utils import to_categorical
sequences = [to_categorical(x, num_classes=vocab_size) for x in X]
X = np.asarray(sequences)
y = to_categorical(y, num_classes=vocab_size)
print(X.shape)
print(y.shape)

Total Sequences: 3436
['Cama', 'amag', 'maga', 'agas', 'gast', 'ast ', 'st  ', 't   ', 'Yone', 'onef', 'nefe', 'efec', 'fect', 'ect ', 'ct  ', 't   ', 'Kome', 'omen', 'meni', 'enit', 'nit ', 'it  ', 't   ', 'Jere', 'erex', 'rex ', 'ex  ', 'x   ', 'Pevo', 'evon', 'voni', 'oni ', 'ni  ', 'i   ', 'Fose', 'oset', 'setr', 'etro', 'tro ', 'ro  ', 'o   ', 'Sled', 'ledr', 'edri', 'drin', 'rin ', 'in  ', 'n   ', 'Clim', 'limi', 'imin', 'minh', 'inho', 'nho ', 'ho  ', 'o   ', 'Yqon', 'qone', 'ones', 'nes ', 'es  ', 's   ', 'Aone', 'ones', 'nes ', 'es  ', 's   ', 'Zrek', 'reke', 'eker', 'kera', 'eraw', 'rawi', 'awin', 'win ', 'in  ', 'n   ', 'Tara', 'aras', 'rast', 'astr', 'stra', 'tra ', 'ra  ', 'a   ', 'Sust', 'usti', 'stio', 'tion', 'ion ', 'on  ', 'n   ', 'Uria', 'rian', 'iano', 'ano ', 'no  ', 'o   ', 'Glon', 'lon ', 'on  ', 'n   ', 'Hasa', 'asar', 'sari', 'arid', 'rid ', 'id  ', 'd   ', 'Vlar', 'lard', 'ard ', 'rd  ', 'd   ', 'Yeve', 'eveq', 'veqi', 'eqio', 'qion', 'ion ', 'on  ', 'n   ', '

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,LSTM, SimpleRNN, GRU
from tensorflow.keras.utils import plot_model
from keras.layers import Bidirectional

def define_bidirectional_model(X):
  model = Sequential()
  model.add(Bidirectional(GRU(75, input_shape=(X.shape[1], X.shape[2]))))
  model.add(Dense(vocab_size, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  model.build(input_shape=(None, X.shape[1], X.shape[2]))
  model.summary()
  plot_model(model, to_file='bidirectional_model.png', show_shapes=True)
  return model

model = define_bidirectional_model(X)
'''
def define_model(X):
  model = Sequential()
  model.add(LSTM(75, input_shape=(X.shape[1], X.shape[2])))
  model.add(Dense(vocab_size, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
  model.summary()
  plot_model(model, to_file='model.png', show_shapes=True)
  return model

model = define_model(X)
'''
import pickle
model.fit(X, y, epochs=100, verbose=2)

model.save('BRNN.keras')
pickle.dump(mapping, open('mapping_names.pkl','wb'))

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_11 (Bidirect  (None, 150)               58500     
 ional)                                                          
                                                                 
 dense_16 (Dense)            (None, 53)                8003      
                                                                 
Total params: 66503 (259.78 KB)
Trainable params: 66503 (259.78 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/100
94/94 - 4s - loss: 2.7376 - accuracy: 0.4083 - 4s/epoch - 47ms/step
Epoch 2/100
94/94 - 1s - loss: 1.9161 - accuracy: 0.4443 - 523ms/epoch - 6ms/step
Epoch 3/100
94/94 - 0s - loss: 1.8207 - accuracy: 0.4783 - 484ms/epoch - 5ms/step
Epoch 4/100
94/94 - 1s - loss: 1.7351 - accuracy: 0.4970 - 518ms/epoch - 6ms/step
Epoch 5/100
94/94 -

In [ ]:
'''
def generate_seq_with_temperature(model, mapping, seq_length, seed_text, n_chars, temperature=1.0):
  in_text = seed_text
  for _ in range(n_chars):
    encoded = [mapping[char] for char in in_text]
    encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
    encoded = to_categorical(encoded, num_classes=len(mapping))
    encoded = encoded.reshape(1, seq_length, len(mapping))

    yhat_probs = model.predict(encoded, verbose=0)
    yhat_probs = yhat_probs[0]  # Get the probabilities for the next character
    yhat_probs = yhat_probs ** (1.0 / temperature)  # Apply temperature

    # Normalize probabilities to ensure they sum to 1
    yhat_probs /= yhat_probs.sum()

    # Sample the next character based on probabilities
    yhat = np.random.choice(len(mapping), p=yhat_probs)

    out_char = ''
    for char, index in mapping.items():
        if index == yhat:
            out_char = char
            break

    in_text += out_char
  return in_text
'''
def generate_seq_with_temperature_and_seed(model, mapping, seq_length, seed_text, n_chars, temperature=1.6, random_seed=666):
    if random_seed is not None:
        np.random.seed(random_seed)

    in_text = seed_text
    for _ in range(n_chars):
        encoded = [mapping[char] for char in in_text]
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        encoded = to_categorical(encoded, num_classes=len(mapping))
        encoded = encoded.reshape(1, seq_length, len(mapping))

        yhat_probs = model.predict(encoded, verbose=0)
        yhat_probs = yhat_probs[0]  # Get the probabilities for the next character
        yhat_probs = yhat_probs ** (1.0 / temperature)  # Apply temperature

        # Normalize probabilities to ensure they sum to 1
        yhat_probs /= yhat_probs.sum()

        # Sample the next character based on probabilities
        yhat = np.random.choice(len(mapping), p=yhat_probs)

        out_char = ''
        for char, index in mapping.items():
            if index == yhat:
                out_char = char
                break

        in_text += out_char

    return in_text

# Example usage with temperature=0.8
print(generate_seq_with_temperature(model, mapping, 3, 'A', 7 ))
print(generate_seq_with_temperature(model, mapping, 3, 'B', 7))
print(generate_seq_with_temperature(model, mapping, 3, 'C', 7))
print(generate_seq_with_temperature(model, mapping, 3, 'D', 7))
print(generate_seq_with_temperature(model, mapping, 3, 'E', 7))
print(generate_seq_with_temperature(model, mapping, 3, 'F', 7))
print(generate_seq_with_temperature(model, mapping, 3, 'G', 7))
print(generate_seq_with_temperature(model, mapping, 3, 'H', 7))
print(generate_seq_with_temperature(model, mapping, 3, 'I', 7))
print(generate_seq_with_temperature(model, mapping, 3, 'J', 7))
print(generate_seq_with_temperature(model, mapping, 3, 'K', 7))
print(generate_seq_with_temperature(model, mapping, 3, 'L', 7))
print(generate_seq_with_temperature(model, mapping, 3, 'M', 7))
print(generate_seq_with_temperature(model, mapping, 3, 'N', 7))
print(generate_seq_with_temperature(model, mapping, 3, 'O', 7))
print(generate_seq_with_temperature(model, mapping, 3, 'P', 7))
print(generate_seq_with_temperature(model, mapping, 3, 'R', 7))
print(generate_seq_with_temperature(model, mapping, 3, 'S', 7))
print(generate_seq_with_temperature(model, mapping, 3, 'T', 7))
print(generate_seq_with_temperature(model, mapping, 3, 'U', 7))
print(generate_seq_with_temperature(model, mapping, 3, 'V', 7))
print(generate_seq_with_temperature(model, mapping, 3, 'Y', 7))
print(generate_seq_with_temperature(model, mapping, 3, 'Z', 7))
print(generate_seq_with_temperature(model, mapping, 3, 'W', 7))
print(generate_seq_with_temperature(model, mapping, 3, 'Q', 7))





Anos    
Blomenit
Canbe   
Dnitlen 
Eriano  
Fones   
Glfek   
Host    
Iver    
Jene    
Kgron   
Lplomeni
Mtern   
Nteget  
Oluntave
Pihomene
RSvergan
Spromers
Tunselar
Uinacca 
VHlgak  
Ynete   
Zokk    
Wenzeron
Qadelp  


In [ ]:
from keras.models import load_model
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
import numpy as np

def generate_seq(model, mapping, seq_length, seed_text, n_chars):
  in_text = seed_text
  # generate a fixed number of characters
  for _ in range(n_chars):
    # encode the characters as integers
    encoded = [mapping[char] for char in in_text]
    # truncate sequences to a fixed length
    encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')

    encoded = to_categorical(encoded, num_classes=len(mapping))
    encoded = encoded.reshape(1, seq_length, len(mapping))

    yhat = model.predict(encoded, verbose=0)
    yhat = np.argmax(yhat)  # Get the index of the highest probabilityfrom pickle import load


    out_char = ''
    for char, index in mapping.items():
      if index == yhat:
        out_char = char
        break
    # append to input
    in_text += out_char
  return in_text

model = load_model('model_names.keras')
mapping = load(open('mapping_names.pkl', 'rb'))
print(generate_seq(model, mapping, 3, 'A', 11))
print(generate_seq(model, mapping, 3, 'C', 11))
print(generate_seq(model, mapping, 3, 'G', 11))
print(generate_seq(model, mapping, 3, 'Y', 11))
print(generate_seq(model, mapping, 3, 'R', 11))
print(generate_seq(model, mapping, 3, 'T', 11))
print(generate_seq(model, mapping, 3, 'E', 11))
print(generate_seq(model, mapping, 3, 'V', 11))
print(generate_seq(model, mapping, 3, 'L', 11))

Aone        
Cont        
Gerhan      
Yone        
Reritino    
Telast      
Eone        
Vold        
Lerax       


In [ ]:
from pickle import load
from keras.models import load_model
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
import numpy as np

def generate_seq(model, mapping, seq_length, seed_text, n_chars):
  in_text = seed_text
  for _ in range(n_chars):
    encoded = [mapping[char] for char in in_text]
    encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
    encoded = to_categorical(encoded, num_classes=len(mapping))
    encoded = encoded.reshape(1, seq_length, len(mapping))  # Corrected reshaping
    yhat_probs = model.predict(encoded, verbose=0)
    predicted_class = np.argmax(yhat_probs)
    predicted_char = next(char for char, index in mapping.items() if index == predicted_class)
    in_text += predicted_char
  return in_text

# Example usage:
# Load your model, mapping, and other necessary variables
# generated_text = generate_seq(model, mapping, seq_length, seed_text, n_chars)


In [ ]:
import keras.backend as K

def dice_coef(y_true, y_pred, smooth=1e-5, threshold=0.5):
    y_pred = K.cast(y_pred > threshold, dtype=tf.float32)
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    dice = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return dice

def dice_loss(smooth=1e-5, threshold=0.5):
    def loss(y_true, y_pred):
        return -dice_coef(y_true, y_pred, smooth, threshold)
    return loss

import torch
import torch.nn as nn

class CustomLoss(nn.Module):
    def __init__(self):
        super(CustomLoss, self).__init__()

    def forward(self, inputs, targets):
        loss = -1 * (targets * torch.log(inputs) + (1 - targets) * torch.log(1 - inputs))
        return loss.mean()

